# 1st stage: LLM-as-a-Judge
## Data Pre-processing

In [ ]:
import os
import time
import pandas as pd
import numpy as np
import scipy.stats as stats
from datetime import datetime
from openai import OpenAI

In [ ]:
BASE_PATH = "../data/"

In [ ]:
description_df = pd.read_csv(BASE_PATH + "raw/1-2_RS_2024_基本情報_事業概要等.zip")
kpi_df = pd.read_csv(BASE_PATH + "raw/3-1_RS_2024_効果発現経路_目標・実績.zip")
check_df = pd.read_csv(BASE_PATH + "raw/4-1_RS_2024_点検・評価.zip")

In [ ]:
kpi_df = kpi_df.rename(
    columns={
        "種別（アクティビティ・アウトプット・アウトカム）": "kpi_type",
        "アクティビティ／活動目標／成果目標": "kpi_description",
        "アウトカムの期間": "outcome_term",
        "活動指標／成果指標": "kpi",
    }
)

kpi_df = kpi_df.drop_duplicates(
    subset=["予算事業ID", "kpi_type", "kpi_description", "outcome_term", "kpi"]
)

In [ ]:
kpi_df.query("予算事業ID == 6080")[["kpi_type", "kpi_description", "outcome_term", "kpi"]]

In [ ]:
print(kpi_df["kpi_type"].unique())
print(kpi_df["outcome_term"].unique())
kpi_df["outcome_term"] = kpi_df["outcome_term"].fillna("3.長期")

In [ ]:
output = (
    kpi_df.query("kpi_type== 'アウトプット'")
    .groupby(["予算事業ID"])["kpi_description"]
    .apply(lambda x: "/".join(sorted(list(x))))
)
short_outcome = (
    kpi_df.query("kpi_type== 'アウトカム' and outcome_term == '1.短期'")
    .groupby(["予算事業ID"])["kpi"]
    .apply(lambda x: "/".join(sorted(list(x))))
)  # kpi
mid_outcome = (
    kpi_df.query("kpi_type== 'アウトカム' and outcome_term == '2.中期'")
    .groupby(["予算事業ID"])["kpi"]
    .apply(lambda x: "/".join(sorted(list(x))))
)  # kpi
long_outcome = (
    kpi_df.query("kpi_type== 'アウトカム' and outcome_term == '3.長期'")
    .groupby(["予算事業ID"])["kpi"]
    .apply(lambda x: "/".join(sorted(list(x))))
)  # kpi

In [ ]:
fixed_kpi_df = pd.concat([output, short_outcome, mid_outcome, long_outcome], axis=1)
fixed_kpi_df.columns = ["output", "short_outcome", "mid_outcome", "long_outcome"]
fixed_kpi_df["outcomes"] = fixed_kpi_df.apply(
    lambda x: "[short_outcome]"
    + str(x["short_outcome"])
    + "[mid_outcome]"
    + str(x["mid_outcome"])
    + "[long_outcome]"
    + str(x["long_outcome"]),
    axis=1,
)
fixed_kpi_df = fixed_kpi_df.reset_index()

In [ ]:
dtm = description_df[
    [
        "事業年度",
        "予算事業ID",
        "事業名",
        "政策所管府省庁",
        "事業の目的",
        "現状・課題",
        "事業の概要",
        "事業開始年度",
        "事業終了（予定）年度",
        "主要経費",
    ]
].merge(fixed_kpi_df, on=["予算事業ID"])
print("レコード数: ", len(dtm))

In [ ]:
dtm.columns = [
    "fiscal_year",
    "budget_project_id",
    "project_name",
    "ministry",
    "project_purpose",
    "current_situation_and_issues",
    "project_summary",
    "project_start_year",
    "project_end_year_estimated",
    "main_expenses",
    "output",
    "short_outcome",
    "mid_outcome",
    "long_outcome",
    "outcomes",
]

In [ ]:
dtm.groupby("ministry")["budget_project_id"].count().sort_values(ascending=False)

In [ ]:
dtm.groupby("main_expenses")["budget_project_id"].count().sort_values(ascending=False)

In [ ]:
# 事務経費や経済協力費は除いた、100以上ある品目ベースでしらべる
target_expenses = [
    _expenses
    for _expenses in dtm.groupby("main_expenses")[["budget_project_id"]]
    .count()
    .query("budget_project_id >= 100")
    .index
    if _expenses not in ["その他の事項経費", "経済協力費"]
]
target_expenses

selected_dtm = dtm.query("main_expenses in @target_expenses")
len(selected_dtm)

In [ ]:
selected_dtm.pivot_table(
    index="ministry",
    columns="main_expenses",
    values="budget_project_id",
    aggfunc="count",
).fillna(0)

In [ ]:
plan_words_list = ["実証", "PoC", "poc", "POC", "POC", "概念実証", "調査", "調査事業"]
pattern = "|".join(plan_words_list)
selected_dtm["plan_project"] = (
    selected_dtm["output"].str.contains(pattern, case=False, na=False).astype(int)
)

In [ ]:
selected_dtm.pivot_table(
    index="ministry",
    columns="plan_project",
    values="budget_project_id",
    aggfunc="count",
).fillna(0)

In [ ]:
selected_dtm.pivot_table(
    index="main_expenses",
    columns="plan_project",
    values="budget_project_id",
    aggfunc="count",
).fillna(0)

In [ ]:
# サンプルチェック
selected_dtm.query("budget_project_id == 6080").T

## LLM score

In [ ]:
client = OpenAI(
    api_key= os.environ.get('OPENAI_API_KEY'),
)

In [ ]:
content_text = """
# 役割
あなたはとても優秀な日本の国家公務員です。各省予算事業を評価する立場にあります。\n
必ず厳しく評価してください。\n
与えられたデータをよく読み、指定された観点・基準に基づき評価してください。\n

# タスク
あなたは、「指定された観点・基準」に基づき、5段階で評価してください。\n
5 : 上位5事業（良い）
4 : やや良い
3 : 普通
2 : やや悪い
1 : 下位5事業（悪い）

# 評価基準：Consistency of output and outcomes
- 事業概要を読み、outputに対する設定アウトカムの適切性を評価していないか。
- 低評価 -1：output -> outcomeに飛躍がある場合は減点
- 低評価 -1：outcomeのstep間に飛躍がある場合は減点
- 低評価 -3：無理やりoutcomeを設定していないか？本質的な意味のあるOutcomeでない場合は大きく減点
- 低評価 -3：outputがそもそも課題の解決に直接つながっていない場合は大きく減点
- 評価対象外：客観性や具体性、明確性は問わない（客観的・具体的であっても高得点にしてはいけない）

# 作業手順
## step1 ベンチーマークを読み込む
- 上位5事業（評価=5）と下位5事業（評価＝１）を決める
- また全体の平均レベルを把握する
- 厳しく採点すること
- まず得点の理由を考えて、それに基づいて採点すること

## step2 評価
- step1で確認したベンチーマーク(平均レベル)と「比較して」それぞれに5段階で評価を与えます
- 必ずベンチーマークを意識して、相対的に評価しなさい

# 注意：
- 対象予算事業すべてを評価してください
- 最低でも評価5は5事業、評価1も5事業、必ず選定すること
- 評価３（普通）は極力つかわないようにすること

# 出力はdict ※これ以外は絶対出力しないで！(不足説明は絶対不要)\n
- budget_project_id: 与えられた予算事業IDに対応\n
- point: あなたが付与した点数\n
- reason: 得点の理由\n
# 出力形式 ※これ以外は絶対出力しないで！！\n
[
    {"budget_project_id" : 3503, "point": 4, "reason" : "~~~だから。"},
    {"budget_project_id" : 3533, "point": 1, "reason" : "~~~がよくわからない。"},
]
"""


In [ ]:
def ask_llm_comparison(content_text, dtm_text):
    return client.responses.create(
        model="gpt-4o",
        temperature=0,
        input=[
            {"role": "developer", "content": content_text},
            {
                "role": "user",
                "content": [
                    {
                        "type": "input_text",
                        "text": f"評価しなさい。データ : {dtm_text}",
                    },
                ]
            }
        ]
    )


In [ ]:
result_logic1_list = []
for _expenses in target_expenses:
    selected_dtm_with_exp = (
        selected_dtm.query("main_expenses == @_expenses")
        .sample(frac=1, random_state=0)
        .reset_index(drop=True)
    )
    bin_labels = [0, 1, 2]
    selected_dtm_with_exp["bin"] = pd.qcut(
        selected_dtm_with_exp.index, len(bin_labels), labels=bin_labels
    )
    for _bin in bin_labels:
        dtm_with_class_bin = selected_dtm_with_exp.query("bin == @_bin")
        dtm_text = dtm_with_class_bin[
            ["budget_project_id", "project_summary", "output", "outcomes"]
        ].to_csv(index=False)
        response = ask_llm_comparison(content_text, dtm_text)
        if len(response.output_text.split("json")) > 1:
            logic_point = pd.DataFrame(
                json.loads(
                    response.output_text.split("json")[1]
                    .strip("```json\n")
                    .strip("```")
                )
            )
        else:
            logic_point = pd.DataFrame(
                json.loads(response.output_text.strip("```json\n").strip("```"))
            )
        logic_point.columns = ["budget_project_id", "logic1_point", "logic1_reason"]
        logic_point = dtm_with_class_bin.merge(
            logic_point, on="budget_project_id", how="left"
        )
        print(f"{_expenses}, {len(dtm_with_class_bin)} , {len(logic_point)}")
        print(
            logic_point.groupby("plan_project").agg(
                {"logic1_point": ["count", "mean", "std"]}
            )
        )
        result_logic1_list.append(logic_point)
    time.sleep(0.5)
logic1_point_result = pd.concat(result_logic1_list)

## output

In [ ]:
today_str = datetime.now().strftime("%Y%m%d")
filename = f"{today_str}_pre_logic_point_data.csv"
logic1_point_result.to_csv(BASE_PATH + "dtm/" + filename, index=False)

In [ ]:
logic1_point_result.query("budget_project_id == 6080")

# FYI

In [ ]:
logic1_point_result.groupby("plan_project").agg(
    {"logic1_point": ["count", "mean", "std"]}
)

In [ ]:
logic1_point_result.pivot_table(
    index="plan_project",
    columns="logic1_point",
    values="budget_project_id",
    aggfunc="count",
).fillna(0)

In [ ]:
# カイ2乗検定
chi2, p, dof, expected = stats.chi2_contingency(
    logic1_point_result.pivot_table(
        index="plan_project",
        columns="logic1_point",
        values="budget_project_id",
        aggfunc="count",
    ).fillna(0)
)

# 結果の表示
print(f"カイ2乗統計量: {chi2}")
print(f"p値: {p}")
print(f"自由度: {dof}")
print(f"期待度数: \n{expected}")

In [ ]:
for _expenses in target_expenses:
    _point = logic1_point_result.query("main_expenses == @_expenses")

    # カイ2乗検定
    chi2, p, dof, expected = stats.chi2_contingency(
        _point.pivot_table(
            index="plan_project",
            columns="logic1_point",
            values="budget_project_id",
            aggfunc="count",
        ).fillna(0)
    )

    # 結果の表示
    print(_expenses)
    display(
        _point.groupby("plan_project").agg({"logic1_point": ["count", "mean", "std"]})
    )
    print(f"カイ2乗統計量: {chi2}")
    print(f"p値: {p}")
    print(f"自由度: {dof}")
    print("#" * 20)